#  第一题  

In [1]:
# 标准答案
def read_config(filename):
    """读取配置文件并解析为字典"""
    config = {}

    try:
        with open(filename, "r", encoding="utf-8") as file:
            for line in file:
                print("before strip:", line, end="")
                line = line.strip() # 先去除头尾的字符（默认是空格），可以把换行符去除。
                print("after strip:", line)
                print("=" * 20)
                if line and "=" in line:
                    key, value = line.split("=", 1) # 按照=号截取为一个列表，最多从头开始截取一次，其实我感觉不用写1也可以，因为赋值号只能有一次，但是可能有些字符串包含=号，这么写会健壮一点，只从最开始匹配到的=号开始截取，保险一点。
                    print(f'key is:{key},value is:{value},key的类型:{type(key)},value的类型:{type(value)}')
                    key = key.strip() # 必须要进行strip操作，因为上面的strip只能去除头尾的空格或其他字符序列，中间的不能去除，比如:data = 12,这样key就是:'data ',注意这里的key的结尾有一个空格，value是' 12',12千米也有一个空格，所以要进行strip操作，strip之后，key为'data',value为'12'
                    value = value.strip()

                    # 数据类型转换
                    print('value.isdigit()',value.isdigit())
                    if value.lower() == "true":
                        config[key] = True
                    elif value.lower() == "false":
                        config[key] = False
                    else:
                        try:
                            float_val = float(value)
                            print('float_val is:',float_val)
                            if float_val.is_integer():
                                config[key] = int(value)
                            else:
                                config[key] = float_val
                        except ValueError:
                            config[key] = value
    except FileNotFoundError:
        print(f"配置文件 {filename} 不存在")
        return {}
    except Exception as e:
        print(f"读取配置文件时出错: {e}")
        return {}

    return config


# 创建配置文件
config_content = """
database_host=localhost
database_port=5432
database_name=myapp
debug_mode=True
max_connections=100.6
"""

with open("config.txt", "w", encoding="utf-8") as f:
    f.write(config_content)


# 第二步：编写配置解析器
def parse_config(filename):
    """
    解析配置文件并返回字典
    支持自动类型转换：整数、布尔值、字符串
    """
    config = {}

    # TODO: 在这里实现配置文件读取和解析逻辑
    config = read_config("config.txt")
    print("config is:", config)
    # 提示：
    # 1. 打开文件并逐行读取
    # 2. 解析键值对（key=value格式）
    # 3. 进行类型转换（int、bool、str）

    return config


# 测试代码
if __name__ == "__main__":
    result = parse_config("config.txt")
    print("解析结果：", result)
    print("类型检查：")
    for key, value in result.items():
        print(f"  {key}: {value} (类型: {type(value).__name__})")

before strip: 
after strip: 
before strip: database_host=localhost
after strip: database_host=localhost
key is:database_host,value is:localhost,key的类型:<class 'str'>,value的类型:<class 'str'>
value.isdigit() False
before strip: database_port=5432
after strip: database_port=5432
key is:database_port,value is:5432,key的类型:<class 'str'>,value的类型:<class 'str'>
value.isdigit() True
float_val is: 5432.0
before strip: database_name=myapp
after strip: database_name=myapp
key is:database_name,value is:myapp,key的类型:<class 'str'>,value的类型:<class 'str'>
value.isdigit() False
before strip: debug_mode=True
after strip: debug_mode=True
key is:debug_mode,value is:True,key的类型:<class 'str'>,value的类型:<class 'str'>
value.isdigit() False
before strip: max_connections=100.6
after strip: max_connections=100.6
key is:max_connections,value is:100.6,key的类型:<class 'str'>,value的类型:<class 'str'>
value.isdigit() False
float_val is: 100.6
config is: {'database_host': 'localhost', 'database_port': 5432, 'database_name': '

#### 对于上述标准答案的一些地方的理解
1. 数据类型转换的逻辑
```python
# 数据类型转换
if value.lower() == "true":
    config[key] = True
elif value.lower() == "false":
    config[key] = False
elif value.isdigit():
    config[key] = int(value)
else:
    config[key] = value
```
***你可能会有疑问：为什么要`value.lower()`？为什么要和true和false做比较？python中不是写作True或者False吗？***

解答：其实这里和什么True和False的写法是没有关系的。value.lower()是在将value转换成全小写，看一下转换成全小写后是不是true，这样能提高配置的灵活性，因为我们这里是在做数据类型的转换，配置的代码中可能会写:data = true，或者data = True，极端的甚至写成data = tRuE，我们直接把=右边的value给lower()了一下，这样不管用户写什么大小写混合的布尔值，都能被识别成全小写的布尔值，假如识别到该布尔值的全小写是true,说明当前这个配置项就是布尔类型的，那么config[key] = True，也就是说解析完之后的config被赋值为True,也就是:data = True。

---
***你可能又会有疑问：这里的数据类型转换的逻辑为什么是这样？***

解答：那我来说下这个转换的逻辑过程，基础数据类型有int,str,bool,float

很明显，前两个if判断是用来判断当value为bool类型的时候，当value为纯数字的时候，将它改为int类型（这里不能判断浮点类型，稍后改进），当既不是bool，又不是数字的时候，就直接返回它的类型。

***改进：新增对浮点数的判定，让程序更加健壮***
```python
# 数据类型转换
if value.lower() == "true":
    config[key] = True
elif value.lower() == "false":
    config[key] = False
else:
    try:
        if float(value).is_integer(): # 如果是一个整数。对的，判断是否是整数需要先转换为float才行
            config[key] = int(value)
        else:
            config[key] = float(value)
    except ValueError:
        config[key] = value
```
这样就可以正确处理整数和浮点数了


# 第二题

### 📋 用户实现问题分析

**我的实现中存在以下问题：**

1. **`analyze_log_levels()`** - `KeyError原因：字典中还没有该key时第一次访问会报错`，我一直搞错这个，要谨记‼️

`关于字典的一些补充：`

在 Python 中，当你直接通过 log_dict[key] 访问或设置一个不存在的键时，确实会引发 KeyError 异常。这是因为字典的 `__getitem__` 方法（即 [] 操作符）在设计上要求键必须存在，否则就抛出异常。

而 log_dict.get(key, default) 方法则不同，它是专门设计来处理键可能不存在的情况的。它的工作原理是：
- 如果键存在，返回对应的值
- 如果键不存在，返回你指定的默认值（如果不指定默认值，则返回 None）

所以当你使用 log_dict.get(key, 0) 时：
- 如果 key 存在，返回它的值
- 如果 key 不存在，返回 0 而不会报错

这实际上是两种不同的访问策略：

- [] 操作符：严格要求键必须存在，适合当你确定键存在时使用
- .get() 方法：宽松访问，适合键可能不存在的情况
2. **`get_error_messages()`** - 逻辑错误：返回的是非ERROR日志，应该返回ERROR日志  
3. **`extract_user_ids()`** - 返回嵌套列表，应该展平为简单列表
4. **`hourly_log_count()`** - 小时范围错误：应该是0-23，不是1-24
5. **`load_logs()`** - split(" ")不够健壮，日志消息可能有多个空格

下面是标准答案实现：

In [8]:
# 题目2：日志文件分析器 - 标准答案版本

import re
from collections import defaultdict, Counter
from datetime import datetime

# 重新创建日志文件（确保数据一致）
log_data_standard = """
2024-01-15 10:30:25 INFO User login successful: user_id=123
2024-01-15 10:31:10 ERROR Database connection failed: timeout
2024-01-15 10:32:15 INFO User logout: user_id=123
2024-01-15 10:33:20 WARNING High memory usage: 85%
2024-01-15 10:34:05 INFO User login successful: user_id=456
2024-01-15 10:35:30 ERROR Authentication failed: invalid_token
"""

with open("access_standard.log", "w", encoding="utf-8") as f:
    f.write(log_data_standard)


class LogAnalyzerStandard:
    """标准版日志分析器类"""

    def __init__(self, log_file):
        self.log_file = log_file
        self.logs = []
        self.load_logs()

    def load_logs(self):
        """加载日志文件 - 标准实现"""
        try:
            with open(self.log_file, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line:  # 跳过空行
                        # 使用正则表达式分割，更健壮
                        # 格式：日期 时间 级别 消息
                        parts = line.split(
                            " ", 3
                        )  # 最多分割3次，保持消息完整，得到诸如：['2024-01-15', '10:30:25', 'INFO', 'User login successful: user_id=123']，最后的User login xxx: user_id=xxx是一组的
                        # print('parts:',parts)
                        if len(parts) >= 4:
                            self.logs.append(parts)
        except FileNotFoundError:
            print(f"错误：找不到文件 {self.log_file}")
        except Exception as e:
            print(f"读取文件时出错：{e}")

    def analyze_log_levels(self):
        """统计不同日志级别的数量 - 标准实现"""
        # 方法1：使用字典get方法
        log_levels = {}
        # for log in self.logs:
        #     print('log:',log)
        #     if len(log) >= 3:
        #         level = log[2]
        #         log_levels[level] = log_levels.get(level, 0) + 1

        # 方法2：使用Counter（更简洁），Counter的括号中是一个生成器表达式
        """
        为什么Counter括号中是一个生成器表达式？并没有出现yield啊？
        生成器表达式（Generator Expression）是Python中的一种语法糖，它不需要显式使用yield关键字。它的语法类似于列表推导式，但使用圆括号而不是方括号。
        在这个例子中：
        Counter(log[2] for log in self.logs if len(log) >= 3)
        (log[2] for log in self.logs if len(log) >= 3)就是一个生成器表达式，它会按需生成值，而不是一次性创建整个列表。这比使用列表推导式更节省内存，特别是当处理大量数据时。
        生成器表达式和yield的区别在于：
        - yield用于定义生成器函数 
        - 生成器表达式是一种更简洁的语法，用于创建简单的生成器
        两者都能实现惰性计算，但生成器表达式更适用于简单的场景。
        """
        # log_levels = Counter(log[2] for log in self.logs if len(log) >= 3)
        # print("使用了counter的log_levels:", log_levels)
        
        # 方法3：使用defaultdict
        log_levels = defaultdict(int)
        for log in self.logs:
            if len(log) >= 3:
                log_levels[log[2]] += 1

        return dict(log_levels)

    def extract_user_ids(self):
        """提取所有用户ID - 标准实现"""
        user_ids = []
        pattern = r"user_id=(\d+)"  # 使用捕获组直接提取数字,⚠️提取的是()内的内容，会得到小括号内的部分，user_id=123 - > 得到：123。我的答案是：r"user_id=[\d]+"，没有分组捕获，无法单独提取数字部分（因为没有括号），会得到整个匹配文本，比如user_id=123

        for log in self.logs:
            if len(log) >= 4:
                message = log[3]
                matches = re.findall(pattern, message)
                user_ids.extend(matches)  # 直接扩展，避免嵌套列表
                print('user_ids:',user_ids)

        # 去重并保持顺序
        seen = set()
        unique_user_ids = []
        for uid in user_ids:
            if uid not in seen:
                seen.add(uid)
                unique_user_ids.append(uid)

        return unique_user_ids

    def get_error_messages(self):
        """找出所有错误信息 - 标准实现"""
        error_messages = []

        for log in self.logs:
            if len(log) >= 3 and log[2] == "ERROR":  # 正确的条件
                # 重构完整的日志消息
                full_message = " ".join(log)
                error_messages.append(full_message)

        return error_messages

    def hourly_log_count(self):
        """统计每小时的日志条数 - 标准实现"""
        hourly_count = defaultdict(int)

        for log in self.logs:
            if len(log) >= 2:
                try:
                    # 解析时间戳
                    timestamp_str = f"{log[0]} {log[1]}"
                    dt = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
                    hour = dt.hour  # 0-23
                    hourly_count[hour] += 1
                except ValueError as e:
                    print(f"时间解析错误：{e}")

        # 转换为标准字典并补充0计数的小时
        result = {}
        for hour in range(24):  # 0-23小时
            result[hour] = hourly_count[hour]

        return result

    def generate_report(self):
        """生成完整的分析报告"""
        print("=" * 50)
        print("📊 日志分析报告")
        print("=" * 50)

        # 基本信息
        print(f"📁 日志文件：{self.log_file}")
        print(f"📝 总日志条数：{len(self.logs)}")

        # 日志级别统计
        levels = self.analyze_log_levels()
        print(f"\n📈 日志级别统计：")
        for level, count in sorted(levels.items()):
            print(f"   {level}: {count} 条")

        # 用户活动
        user_ids = self.extract_user_ids()
        print(f"\n👥 活跃用户：{len(user_ids)} 个")
        print(f"   用户ID: {', '.join(user_ids)}")

        # 错误信息
        errors = self.get_error_messages()
        print(f"\n❌ 错误日志：{len(errors)} 条")
        for error in errors:
            print(f"   {error}")

        # 时间分布
        hourly = self.hourly_log_count()
        active_hours = [(hour, count) for hour, count in hourly.items() if count > 0]
        print(f"\n⏰ 活跃时段：")
        for hour, count in sorted(active_hours):
            print(f"   {hour:02d}:00 - {count} 条日志")


# 测试标准实现
print("=== 标准答案测试 ===")
analyzer_std = LogAnalyzerStandard("access_standard.log")

print("\n1. 日志级别统计：")
levels = analyzer_std.analyze_log_levels()
for level, count in levels.items():
    print(f"   {level}: {count}")

print("\n2. 用户ID列表：", analyzer_std.extract_user_ids())

print("\n3. 错误信息：")
errors = analyzer_std.get_error_messages()
for error in errors:
    print(f"   {error}")

print("\n4. 每小时日志数量：")
hourly = analyzer_std.hourly_log_count()
for hour, count in hourly.items():
    if count > 0:  # 只显示有日志的小时
        print(f"   {hour:02d}点: {count}条")

# 生成完整报告
print("\n" + "=" * 60)
analyzer_std.generate_report()

=== 标准答案测试 ===

1. 日志级别统计：
   INFO: 3
   ERROR: 2
user_ids: ['123']
user_ids: ['123']
user_ids: ['123', '123']
user_ids: ['123', '123']
user_ids: ['123', '123', '456']
user_ids: ['123', '123', '456']

2. 用户ID列表： ['123', '456']

3. 错误信息：
   2024-01-15 10:31:10 ERROR Database connection failed: timeout
   2024-01-15 10:35:30 ERROR Authentication failed: invalid_token

4. 每小时日志数量：
   10点: 6条

📊 日志分析报告
📁 日志文件：access_standard.log
📝 总日志条数：6

📈 日志级别统计：
   ERROR: 2 条
   INFO: 3 条
user_ids: ['123']
user_ids: ['123']
user_ids: ['123', '123']
user_ids: ['123', '123']
user_ids: ['123', '123', '456']
user_ids: ['123', '123', '456']

👥 活跃用户：2 个
   用户ID: 123, 456

❌ 错误日志：2 条
   2024-01-15 10:31:10 ERROR Database connection failed: timeout
   2024-01-15 10:35:30 ERROR Authentication failed: invalid_token

⏰ 活跃时段：
   10:00 - 6 条日志


### 🔍 对比总结

| 功能 | 你的实现 | 标准实现 | 主要区别 |
|------|----------|----------|----------|
| **load_logs()** | `split(" ")` | `split(" ", 3)` | 标准版限制分割次数，保持消息完整 |
| **analyze_log_levels()** | try-except处理KeyError | `dict.get(key, 0)` | 标准版更简洁，避免异常 |
| **extract_user_ids()** | 返回嵌套列表 | 使用捕获组+extend | 标准版直接提取数字，去重 |
| **get_error_messages()** | ❌ `!= "ERROR"` | ✅ `== "ERROR"` | 你的逻辑相反了 |
| **hourly_log_count()** | 小时1-24 | 小时0-23 | 标准版符合实际时间格式 |

### 💡 学习要点

1. **字典操作**：使用 `dict.get(key, default)` 比 try-except 更简洁
2. **正则表达式**：使用捕获组 `(\d+)` 直接提取需要的部分
3. **列表操作**：使用 `extend()` 而不是 `append()` 来展平列表
4. **逻辑条件**：仔细理解需求，避免条件写反
5. **时间处理**：注意小时是0-23，不是1-24
6. **错误处理**：添加适当的异常处理，提高代码健壮性

### 🧐 关于标准答案中模块的使用
1. `Counter`: [点击查看](../知识点/python模块/collections/Counter.ipynb)

### 🧐 关于一些不熟悉的用法
1. `分组捕获`: [点击查看](../知识点/正则表达式/关于分组捕获.ipynb)


你的基本思路是正确的，只是在一些细节上需要调整！ 🎯